<a href="https://www.bigdatauniversity.com"><img src="https://ibm.box.com/shared/static/qo20b88v1hbjztubt06609ovs85q8fau.png" width="400px" align="center"></a>

<h1 align="center"><font size="5">RECURRENT NETWORKS and LSTM IN DEEP LEARNING</font></h1>

<h2>Applying Recurrent Neural Networks/LSTM for Language Modeling</h2>
Hello and welcome to this part. In this notebook, we will go over the topic of Language Modelling, and create a Recurrent Neural Network model based on the Long Short-Term Memory unit to train and benchmark on the Penn Treebank dataset. By the end of this notebook, you should be able to understand how TensorFlow builds and executes a RNN model for Language Modelling.

<h2>The Objective</h2>
By now, you should have an understanding of how Recurrent Networks work -- a specialized model to process sequential data by keeping track of the "state" or context. In this notebook, we go over a TensorFlow code snippet for creating a model focused on <b>Language Modelling</b> -- a very relevant task that is the cornerstone of many different linguistic problems such as <b>Speech Recognition, Machine Translation and Image Captioning</b>. For this, we will be using the Penn Treebank dataset, which is an often-used dataset for benchmarking Language Modelling models.


<h2>Table of Contents</h2>
<ol>
    <li><a href="#language_modelling">What exactly is Language Modelling?</a></li>
    <li><a href="#treebank_dataset">The Penn Treebank dataset</a></li>
    <li><a href="#word_embedding">Work Embedding</a></li>
    <li><a href="#building_lstm_model">Building the LSTM model for Language Modeling</a></li>
    <li><a href="#ltsm">LTSM</a></li>
</ol>
<p></p>
</div>
<br>

--------

<a id="language_modelling"></a>
<h2>What exactly is Language Modelling?</h2>
Language Modelling, to put it simply, <b>is the task of assigning probabilities to sequences of words</b>. This means that, given a context of one or a sequence of words in the language the model was trained on, the model should provide the next most probable words or sequence of words that follows from the given sequence of words the sentence. Language Modelling is one of the most important tasks in Natural Language Processing.

<img src="https://ibm.box.com/shared/static/1d1i5gub6wljby2vani2vzxp0xsph702.png" width="1080">
<center><i>Example of a sentence being predicted</i></center>
<br><br>
In this example, one can see the predictions for the next word of a sentence, given the context "This is an". As you can see, this boils down to a sequential data analysis task -- you are given a word or a sequence of words (the input data), and, given the context (the state), you need to find out what is the next word (the prediction). This kind of analysis is very important for language-related tasks such as <b>Speech Recognition, Machine Translation, Image Captioning, Text Correction</b> and many other very relevant problems. 

<img src="https://ibm.box.com/shared/static/az39idf9ipfdpc5ugifpgxnydelhyf3i.png" width="1080">
<center><i>The above example is a schema of an RNN in execution</i></center>
<br><br>
As the above image shows, Recurrent Network models fit this problem like a glove. Alongside LSTM and its capacity to maintain the model's state for over one thousand time steps, we have all the tools we need to undertake this problem. The goal for this notebook is to create a model that can reach <b>low levels of perplexity</b> on our desired dataset.

For Language Modelling problems, <b>perplexity</b> is the way to gauge efficiency. Perplexity is simply a measure of how well a probabilistic model is able to predict its sample. A higher-level way to explain this would be saying that <b>low perplexity means a higher degree of trust in the predictions the model makes</b>. Therefore, the lower perplexity is, the better.

<a id="treebank_dataset"></a>
<h2>The Penn Treebank dataset</h2>
Historically, datasets big enough for Natural Language Processing are hard to come by. This is in part due to the necessity of the sentences to be broken down and tagged with a certain degree of correctness -- or else the models trained on it won't be able to be correct at all. This means that we need a <b>large amount of data, annotated by or at least corrected by humans</b>. This is, of course, not an easy task at all.

The Penn Treebank, or PTB for short, is a dataset maintained by the University of Pennsylvania. It is <i>huge</i> -- there are over <b>four million and eight hundred thousand</b> annotated words in it, all corrected by humans. It is composed of many different sources, from abstracts of Department of Energy papers to texts from the Library of America. Since it is verifiably correct and of such a huge size, the Penn Treebank is commonly used as a benchmark dataset for Language Modelling.

The dataset is divided in different kinds of annotations, such as Piece-of-Speech, Syntactic and Semantic skeletons. For this example, we will simply use a sample of clean, non-annotated words (with the exception of one tag --<code>&lt;unk&gt;</code>
, which is used for rare words such as uncommon proper nouns) for our model. This means that we just want to predict what the next words would be, not what they mean in context or their classes on a given sentence.

<center>Example of text from the dataset we are going to use, <b>ptb.train</b></center>
<br><br>

<div class="alert alert-block alert-info" style="margin-top: 20px">
    <center>the percentage of lung cancer deaths among the workers at the west <code>&lt;unk&gt;</code> mass. paper factory appears to be the highest for any asbestos workers studied in western industrialized countries he said 
 the plant which is owned by <code>&lt;unk&gt;</code> & <code>&lt;unk&gt;</code> co. was under contract with <code>&lt;unk&gt;</code> to make the cigarette filters 
 the finding probably will support those who argue that the U.S. should regulate the class of asbestos including <code>&lt;unk&gt;</code> more <code>&lt;unk&gt;</code> than the common kind of asbestos <code>&lt;unk&gt;</code> found in most schools and other buildings dr. <code>&lt;unk&gt;</code> said</center>
</div>

<a id="word_embedding"></a>
<h2>Word Embeddings</h2><br/>

For better processing, in this example, we will make use of <a href="https://www.tensorflow.org/tutorials/word2vec/"><b>word embeddings</b></a>, which is <b>a way of representing sentence structures or words as n-dimensional vectors (where n is a reasonably high number, such as 200 or 500) of real numbers</b>. Basically, we will assign each word a randomly-initialized vector, and input those into the network to be processed. After a number of iterations, these vectors are expected to assume values that help the network to correctly predict what it needs to -- in our case, the probable next word in the sentence. This is shown to be a very effective task in Natural Language Processing, and is a commonplace practice.
<br><br>
<font size="4"><strong>
$$Vec("Example") = [0.02, 0.00, 0.00, 0.92, 0.30, \ldots]$$
</strong></font>
<br>
Word Embedding tends to group up similarly used words <i>reasonably</i> close together in the vectorial space. For example, if we use T-SNE (a dimensional reduction visualization algorithm) to flatten the dimensions of our vectors into a 2-dimensional space and plot these words in a 2-dimensional space, we might see something like this:

<img src="https://ibm.box.com/shared/static/bqhc5dg879gcoabzhxra1w8rkg3od1cu.png" width="800">
<center><i>T-SNE Mockup with clusters marked for easier visualization</i></center>
<br><br>
As you can see, words that are frequently used together, in place of each other, or in the same places as them tend to be grouped together -- being closer together the higher they are correlated. For example, "None" is pretty semantically close to "Zero", while a phrase that uses "Italy", you could probably also fit "Germany" in it, with little damage to the sentence structure. The vectorial "closeness" for similar words like this is a great indicator of a well-built model.

<hr>
 

We need to import the necessary modules for our code. We need <b><code>numpy</code></b> and <b><code>tensorflow</code></b>, obviously. Additionally, we can import directly the <b><code>tensorflow.models.rnn</code></b> model, which includes the function for building RNNs, and <b><code>tensorflow.models.rnn.ptb.reader</code></b> which is the helper module for getting the input data from the dataset we just downloaded.

If you want to learn more take a look at https://github.com/tensorflow/models/blob/master/tutorials/rnn/ptb/reader.py

In [1]:
import time
import numpy as np
import tensorflow as tf

In [2]:
!mkdir data
!wget -q -O data/ptb.zip https://ibm.box.com/shared/static/z2yvmhbskc45xd2a9a4kkn6hg4g4kj5r.zip
!unzip -o data/ptb.zip -d data
!cp data/ptb/reader.py .

import reader

mkdir: cannot create directory ‘data’: File exists
Archive:  data/ptb.zip
  inflating: data/ptb/reader.py      
  inflating: data/__MACOSX/ptb/._reader.py  
  inflating: data/__MACOSX/._ptb     


<a id="building_lstm_model"></a>
<h2>Building the LSTM model for Language Modeling</h2>
Now that we know exactly what we are doing, we can start building our model using TensorFlow. The very first thing we need to do is download and extract the <code>simple-examples</code> dataset, which can be done by executing the code cell below.

In [3]:
!wget http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz 
!tar xzf simple-examples.tgz -C data/

--2020-02-29 00:28:22--  http://www.fit.vutbr.cz/~imikolov/rnnlm/simple-examples.tgz
Resolving www.fit.vutbr.cz (www.fit.vutbr.cz)... 147.229.9.23, 2001:67c:1220:809::93e5:917
Connecting to www.fit.vutbr.cz (www.fit.vutbr.cz)|147.229.9.23|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 34869662 (33M) [application/x-gtar]
Saving to: ‘simple-examples.tgz’

simple-examples.tgz 100%[===================>]  33.25M  4.03MB/s    in 9.5s    

2020-02-29 00:28:32 (3.49 MB/s) - ‘simple-examples.tgz’ saved [34869662/34869662]



Additionally, for the sake of making it easy to play around with the model's hyperparameters, we can declare them beforehand. Feel free to change these -- you will see a difference in performance each time you change those!  


In [3]:
#Initial weight scale
init_scale = 0.1
#Initial learning rate
learning_rate = 1.0
#Maximum permissible norm for the gradient (For gradient clipping -- another measure against Exploding Gradients)
max_grad_norm = 5
#The number of layers in our model
num_layers = 2
#The total number of recurrence steps, also known as the number of layers when our RNN is "unfolded"
num_steps = 20
#The number of processing units (neurons) in the hidden layers
hidden_size_l1 = 256
hidden_size_l2 = 128
#The maximum number of epochs trained with the initial learning rate
max_epoch_decay_lr = 4
#The total number of epochs in training
max_epoch = 15
#The probability for keeping data in the Dropout Layer (This is an optimization, but is outside our scope for this notebook!)
#At 1, we ignore the Dropout Layer wrapping.
keep_prob = 1
#The decay for the learning rate
decay = 0.5
#The size for each batch of data
batch_size = 60
#The size of our vocabulary
vocab_size = 10000
embeding_vector_size = 200
#Training flag to separate training from testing
is_training = 1
#Data directory for our dataset
data_dir = "data/simple-examples/data/"

Some clarifications for LSTM architecture based on the arguments:

Network structure:
<ul>
    <li>In this network, the number of LSTM cells are 2. To give the model more expressive power, we can add multiple layers of LSTMs to process the data. The output of the first layer will become the input of the second and so on.
    </li>
    <li>The recurrence steps is 20, that is, when our RNN is "Unfolded", the recurrence step is 20.</li>   
    <li>the structure is like:
        <ul>
            <li>200 input units -> [200x200] Weight -> 200 Hidden units (first layer) -> [200x200] Weight matrix  -> 200 Hidden units (second layer) ->  [200] weight Matrix -> 200 unit output</li>
        </ul>
    </li>
</ul>
<br>

Input layer: 
<ul>
    <li>The network has 200 input units.</li>
    <li>Suppose each word is represented by an embedding vector of dimensionality e=200. The input layer of each cell will have 200 linear units. These e=200 linear units are connected to each of the h=200 LSTM units in the hidden layer (assuming there is only one hidden layer, though our case has 2 layers).
    </li>
    <li>The input shape is [batch_size, num_steps], that is [60x20]. It will turn into [60x20x200] after embedding, and then 20x[60x200]
    </li>
</ul>
<br>

Hidden layer:
<ul>
    <li>Each LSTM has 200 hidden units which is equivalent to the dimensionality of the embedding words and output.</li>
</ul>
<br>

There is a lot to be done and a ton of information to process at the same time, so go over this code slowly. It may seem complex at first, but if you try to apply what you just learned about language modelling to the code you see, you should be able to understand it.

This code is adapted from the <a href="https://github.com/tensorflow/models">PTBModel</a> example bundled with the TensorFlow source code.


<h4>Train data</h4>
The story starts from data: <b>NOTE: The batch size is now 60</b>
<ul>
    <li>Train data is a list of words, of size 929589, represented by numbers, e.g. [9971, 9972, 9974, 9975,...]</li>
    <li>We read data as mini-batch of size b=30. Assume the size of each sentence is 20 words (num_steps = 20). Then it will take $$floor(\frac{N}{b \times h})+1=1548$$ iterations for the learner to go through all sentences once. Where N is the size of the list of words, b is batch size, andh is size of each sentence. So, the number of iterators is 1548
    </li>
    <li>Each batch data is read from train dataset of size 600, and shape of [30x20]</li>
</ul>

First we start an interactive session:

In [4]:
session = tf.InteractiveSession()

In [5]:
# Reads the data and separates it into training data, validation data and testing data
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, vocab, word_to_id = raw_data

In [15]:
def id_to_word(id_list):
    line = []
    for w in id_list:
        for word, wid in word_to_id.items():
            if wid == w:
                line.append(word)
    return line            
                

print(id_to_word(train_data[0:100]))

['aer', 'banknote', 'berlitz', 'calloway', 'centrust', 'cluett', 'fromstein', 'gitano', 'guterman', 'hydro-quebec', 'ipo', 'kia', 'memotec', 'mlx', 'nahb', 'punts', 'rake', 'regatta', 'rubens', 'sim', 'snack-food', 'ssangyong', 'swapo', 'wachter', '<eos>', 'pierre', '<unk>', 'N', 'years', 'old', 'will', 'join', 'the', 'board', 'as', 'a', 'nonexecutive', 'director', 'nov.', 'N', '<eos>', 'mr.', '<unk>', 'is', 'chairman', 'of', '<unk>', 'n.v.', 'the', 'dutch', 'publishing', 'group', '<eos>', 'rudolph', '<unk>', 'N', 'years', 'old', 'and', 'former', 'chairman', 'of', 'consolidated', 'gold', 'fields', 'plc', 'was', 'named', 'a', 'nonexecutive', 'director', 'of', 'this', 'british', 'industrial', 'conglomerate', '<eos>', 'a', 'form', 'of', 'asbestos', 'once', 'used', 'to', 'make', 'kent', 'cigarette', 'filters', 'has', 'caused', 'a', 'high', 'percentage', 'of', 'cancer', 'deaths', 'among', 'a', 'group', 'of']


Lets just read one mini-batch now and feed our network:

In [16]:
itera = reader.ptb_iterator(train_data, batch_size, num_steps)
first_touple = itera.__next__()
x = first_toupl`e[0]
y = first_touple[1]

In [17]:
x.shape

(60, 20)

Lets look at 3 sentences of our input x:

In [23]:
x[0:3]

array([[9970, 9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984,
        9986, 9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995],
       [ 901,   33, 3361,    8, 1279,  437,  597,    6,  261, 4276, 1089,
           8, 2836,    2,  269,    4, 5526,  241,   13, 2420],
       [2654,    6,  334, 2886,    4,    1,  233,  711,  834,   11,  130,
         123,    7,  514,    2,   63,   10,  514,    8,  605]],
      dtype=int32)

we define 2 place holders to feed them with mini-batchs, that is x and y:

In [26]:
_input_data = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]
_targets = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]

Lets define a dictionary, and use it later to feed the placeholders with our first mini-batch:

In [29]:
feed_dict = {_input_data:x, _targets:y}

For example, we can use it to feed <code>\_input\_data</code>:

In [30]:
session.run(_input_data, feed_dict)

array([[9970, 9971, 9972, ..., 9993, 9994, 9995],
       [ 901,   33, 3361, ...,  241,   13, 2420],
       [2654,    6,  334, ...,  514,    8,  605],
       ...,
       [7831,   36, 1678, ...,    4, 4558,  157],
       [  59, 2070, 2433, ...,  400,    1, 1173],
       [2097,    3,    2, ..., 2043,   23,    1]], dtype=int32)

In this step, we create the stacked LSTM, which is a 2 layer LSTM network:

In [33]:
lstm_cell_l1 = tf.contrib.rnn.BasicLSTMCell(hidden_size_l1, forget_bias=0.0)
lstm_cell_l2 = tf.contrib.rnn.BasicLSTMCell(hidden_size_l2, forget_bias=0.0)
stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell_l1, lstm_cell_l2])

Also, we initialize the states of the nework:

<h4>_initial_state</h4>

For each LCTM, there are 2 state matrices, c\_state and m\_state.  c_state and m_state represent "Memory State" and "Cell State". Each hidden layer, has a vector of size 30, which keeps the states. so, for 200 hidden units in each LSTM, we have a matrix of size [30x200]

In [35]:
_initial_state = stacked_lstm.zero_state(batch_size, tf.float32)
_initial_state

(LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros:0' shape=(60, 256) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState/zeros_1:0' shape=(60, 256) dtype=float32>),
 LSTMStateTuple(c=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros:0' shape=(60, 128) dtype=float32>, h=<tf.Tensor 'MultiRNNCellZeroState/BasicLSTMCellZeroState_1/zeros_1:0' shape=(60, 128) dtype=float32>))

Lets look at the states, though they are all zero for now:

In [71]:
session.run(_initial_state)

(LSTMStateTuple(c=array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), h=array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32)),
 LSTMStateTuple(c=array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.]], dtype=float32), h=array([[0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        [0., 0., 0., ..., 0., 0., 0.],
        ...,
        [0., 0., 0., ..., 0., 0., 0.],
  

<h3>Embeddings</h3>
We have to convert the words in our dataset to vectors of numbers. The traditional approach is to use one-hot encoding method that is usually used for converting categorical values to numerical values. However, One-hot encoded vectors are high-dimensional, sparse and in a big dataset, computationally inefficient. So, we use word2vec approach. It is, in fact, a layer in our LSTM network, where the word IDs will be represented as a dense representation before feeding to the LSTM. 

The embedded vectors also get updated during the training process of the deep neural network.
We create the embeddings for our input data. <b>embedding_vocab</b> is matrix of [10000x200] for all 10000 unique words.

In [48]:
embedding_vocab = tf.get_variable("embedding_vocab", [vocab_size, embeding_vector_size])  #[10000x200]

<b>embedding_lookup()</b> finds the embedded values for our batch of 60x20 words. It  goes to each row of <code>input_data</code>, and for each word in the row/sentence, finds the correspond vector in <code>embedding_dic<code>. <br>
It creates a [60x20x200] tensor, so, the first element of <b>inputs</b> (the first sentence), is a matrix of 20x200, which each row of it, is vector representing a word in the sentence.

In [51]:
# Define where to get the data for our embeddings from. I.e. construct the embedded input data
inputs = tf.nn.embedding_lookup(embedding_vocab, _input_data)  #shape=(60, 20, 200) 
inputs

<tf.Tensor 'embedding_lookup:0' shape=(60, 20, 200) dtype=float32>

<h3>Constructing Recurrent Neural Networks</h3>
<b>tf.nn.dynamic_rnn()</b> creates a recurrent neural network using <b>stacked_lstm</b>. 

The input should be a Tensor of shape: [batch_size, max_time, embedding_vector_size], in our case it would be (30, 20, 200)

This method, returns a pair (outputs, new_state) where:
<ul>
    <li><b>outputs</b>: is a length T list of outputs (one for each input), or a nested tuple of such elements.</li>
    <li><b>new_state</b>: is the final state.</li>
</ul>

In [62]:
outputs, new_state =  tf.nn.dynamic_rnn(stacked_lstm, inputs, initial_state=_initial_state)

so, lets look at the outputs. The output of the stackedLSTM comes from 200 hidden_layer, and in each time step(=20), one of them get activated. we use the linear activation to map the 200 hidden layer to a [?x10 matrix]

In [92]:
outputs

<tf.Tensor 'rnn/transpose_1:0' shape=(60, 20, 128) dtype=float32>

In [93]:
session.run(tf.global_variables_initializer())
session.run(outputs[0], feed_dict)

array([[-2.4285330e-04, -6.0026121e-04,  2.4645678e-05, ...,
         1.6524621e-04,  1.3676115e-04, -1.2549039e-04],
       [-2.2214827e-04, -1.1182525e-03,  1.1112423e-05, ...,
         3.4447538e-04,  1.2277138e-04, -2.6792547e-04],
       [-3.5605900e-04, -5.9629395e-04, -2.2696947e-04, ...,
        -9.9164845e-07, -4.1214569e-04, -8.6327933e-04],
       ...,
       [ 4.5237856e-04,  3.0043343e-04, -9.3792041e-04, ...,
        -1.9582795e-04,  3.9315945e-04,  1.9138180e-04],
       [-2.1025212e-04,  3.6966521e-04, -5.3023105e-04, ...,
        -1.2035016e-04,  1.9544056e-04, -1.7368924e-04],
       [-7.6225813e-04,  2.6290570e-04, -9.8913931e-04, ...,
         1.1628195e-05,  2.2953846e-04, -6.0415547e-04]], dtype=float32)

we need to flatten the outputs to be able to connect it softmax layer. Lets reshape the output tensor from  [60 x 20 x 200] to [1200 x 200].

<b>Notice:</b> Imagine our output is 3-d tensor as following (of course each <code>sen_x_word_y</code> is a an embedded vector by itself): 
<ul>
    <li>sentence 1: [[sen1word1], [sen1word2], [sen1word3], ..., [sen1word20]]</li> 
    <li>sentence 2: [[sen2word1], [sen2word2], [sen2word3], ..., [sen2word20]]</li>   
    <li>sentence 3: [[sen3word1], [sen3word2], [sen3word3], ..., [sen3word20]]</li>  
    <li>...  </li>
    <li>sentence 60: [[sen60word1], [sen60word2], [sen60word3], ..., [sen60word20]]</li>   
</ul>
Now, the flatten would convert this 3-dim tensor to:

[ [sen1word1], [sen1word2], [sen1word3], ..., [sen1word20],[sen2word1], [sen2word2], [sen2word3], ..., [sen2word20], ..., [sen60word20] ]


In [96]:
output = tf.reshape(outputs, [-1, hidden_size_l2])
output

<tf.Tensor 'Reshape:0' shape=(1200, 128) dtype=float32>

<h3>logistic unit</h3>
Now, we create a logistic unit to return the probability of the output word in our vocabulary with 10000 words. 

$$Softmax = [1200 \times 200] * [200 \times 10000] + [1 \times 10000] \Longrightarrow [1200 \times 10000]$$

In [100]:
softmax_w = tf.get_variable("softmax_w", [hidden_size_l2, vocab_size]) #[200x10000]
softmax_b = tf.get_variable("softmax_b", [vocab_size]) #[1x10000]
logits = tf.matmul(output, softmax_w) + softmax_b
prob = tf.nn.softmax(logits)

Lets look at the probability of observing words for t=0 to t=20:

In [102]:
session.run(tf.global_variables_initializer())
output_words_prob = session.run(prob, feed_dict)
print("shape of the output: ", output_words_prob.shape)
print("The probability of observing words in t=0 to t=20", output_words_prob[0:20])

shape of the output:  (1200, 10000)
The probability of observing words in t=0 to t=20 [[9.96447852e-05 9.94184666e-05 1.01675003e-04 ... 9.91598572e-05
  9.94456059e-05 9.84428334e-05]
 [9.96456438e-05 9.94225993e-05 1.01661048e-04 ... 9.91590568e-05
  9.94474249e-05 9.84391518e-05]
 [9.96460294e-05 9.94307993e-05 1.01669575e-04 ... 9.91630586e-05
  9.94343864e-05 9.84329745e-05]
 ...
 [9.96505551e-05 9.94182337e-05 1.01670448e-04 ... 9.91634661e-05
  9.94422735e-05 9.84472426e-05]
 [9.96430608e-05 9.94232760e-05 1.01674181e-04 ... 9.91642664e-05
  9.94442598e-05 9.84395738e-05]
 [9.96234448e-05 9.94239745e-05 1.01680867e-04 ... 9.91633424e-05
  9.94433140e-05 9.84347207e-05]]


<h3>Prediction</h3>
What is the word correspond to the probability output? Lets use the maximum probability:

In [105]:
np.argmax(output_words_prob[0:20], axis=1)

array([7033, 2151, 7699, 2151, 6049, 6110, 6961, 6961, 6110,  579, 2244,
       1573, 1573, 1573, 7114, 7114, 2465, 8221, 8221, 9542])

So, what is the ground truth for the first word of first sentence? 

In [109]:
y[0]

array([9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984, 9986,
       9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995, 9996], dtype=int32)

Also, you can get it from target tensor, if you want to find the embedding vector:

In [110]:
targ = session.run(_targets, feed_dict) 
targ[0]

array([9971, 9972, 9974, 9975, 9976, 9980, 9981, 9982, 9983, 9984, 9986,
       9987, 9988, 9989, 9991, 9992, 9993, 9994, 9995, 9996], dtype=int32)

How similar the predicted words are to the target words?

<h4>Objective function</h4>

Now we have to define our objective function, to calculate the similarity of predicted values to ground truth, and then, penalize the model with the error. Our objective is to minimize loss function, that is, to minimize the average negative log probability of the target words:

$$\text{loss} = -\frac{1}{N}\sum_{i=1}^{N} \ln p_{\text{target}_i}$$

This function is already implemented and available in TensorFlow through <b>sequence_loss_by_example</b>. It calculates the weighted cross-entropy loss for <b>logits</b> and the <b>target</b> sequence.  

The arguments of this function are:  
<ul>
    <li>logits: List of 2D Tensors of shape [batch_size x num_decoder_symbols].</li>  
    <li>targets: List of 1D batch-sized int32 Tensors of the same length as logits.</li>   
    <li>weights: List of 1D batch-sized float-Tensors of the same length as logits.</li> 
</ul>

In [118]:
# There we set the weights to all ones
loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(_targets, [-1])],[tf.ones([batch_size * num_steps])])

loss is a 1D batch-sized float Tensor [1200x1]: The log-perplexity for each sequence. Lets look at the first 10 values of loss:

In [126]:
session.run(loss, feed_dict)[:10]

array([9.194739, 9.227651, 9.209609, 9.207912, 9.203433, 9.213476,
       9.220358, 9.19891 , 9.224243, 9.227246], dtype=float32)

Now, we define loss as average of the losses:

In [121]:
cost = tf.reduce_sum(loss) / batch_size
session.run(tf.global_variables_initializer())
session.run(cost, feed_dict)

184.1682

<h3>Training</h3>

To do training for our network, we have to take the following steps:
<ol>
    <li>Define the optimizer.</li>
    <li>Extract variables that are trainable.</li>
    <li>Calculate the gradients based on the loss function.</li>
    <li>Apply the optimizer to the variables/gradients tuple.</li>
</ol>

<h4>1. Define Optimizer</h4>

<b>GradientDescentOptimizer</b> constructs a new gradient descent optimizer. Later, we use constructed <b>optimizer</b> to compute gradients for a loss and apply gradients to variables.

In [128]:
# Create a variable for the learning rate
lr = tf.Variable(0.0, trainable=False)
# Create the gradient descent optimizer with our learning rate
optimizer = tf.train.GradientDescentOptimizer(lr)


<h4>2. Trainable Variables</h4>

Defining a variable, if you passed <i>trainable=True</i>, the variable constructor automatically adds new variables to the graph collection <b>GraphKeys.TRAINABLE_VARIABLES</b>. Now, using <i>tf.trainable_variables()</i> you can get all variables created with <b>trainable=True</b>.

In [129]:
# Get all TensorFlow variables marked as "trainable" (i.e. all of them except _lr, which we just created)
tvars = tf.trainable_variables()
tvars

[<tf.Variable 'embedding_vocab:0' shape=(10000, 200) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0' shape=(456, 1024) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0' shape=(1024,) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0' shape=(384, 512) dtype=float32_ref>,
 <tf.Variable 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0' shape=(512,) dtype=float32_ref>,
 <tf.Variable 'softmax_w:0' shape=(128, 10000) dtype=float32_ref>,
 <tf.Variable 'softmax_b:0' shape=(10000,) dtype=float32_ref>]

In [143]:
# forget gate, information gate, gain gate, output gate are combined as one weight matrix,
# Also,
#   For the 1st LSTM: Hidden state (1, 256), input matrix (1, 200), then first dim of weight is 256+200=456
#   For the 2nd LSTM: Hidden state (1, 128), input maxtrix (1, 256), then 128+256=384
print("LSTM_1 hidden units:", 1024/4)
print("LSTM_2 hidden units:", 512/4)

LSTM_1 hidden units: 256.0
LSTM_2 hidden units: 128.0


Note: we can find the name and scope of all variables:

In [148]:
[v.name for v in tvars]

['embedding_vocab:0',
 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/kernel:0',
 'rnn/multi_rnn_cell/cell_0/basic_lstm_cell/bias:0',
 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/kernel:0',
 'rnn/multi_rnn_cell/cell_1/basic_lstm_cell/bias:0',
 'softmax_w:0',
 'softmax_b:0']

<h4>3. Calculate the gradients based on the loss function</h4>

<h4>Gradient</h4>:
The gradient of a function is the slope of its derivative (line), or in other words, the rate of change of a function. It's a vector (a direction to move) that points in the direction of greatest increase of the function, and calculated by the <b>derivative</b> operation.

First lets recall the gradient function using an toy example:
$$ z = \left(2x^2 + 3xy\right)$$

In [149]:
var_x = tf.placeholder(tf.float32)
var_y = tf.placeholder(tf.float32) 
func_test = 2.0 * var_x * var_x + 3.0 * var_x * var_y
session.run(tf.global_variables_initializer())
session.run(func_test, {var_x:1.0,var_y:2.0})

8.0

The <b>tf.gradients()</b> function allows you to compute the symbolic gradient of one tensor with respect to one or more other tensors—including variables. <b>tf.gradients(func, xs)</b> constructs symbolic partial derivatives of sum of <b>func</b> w.r.t. <i>x</i> in <b>xs</b>. 

Now, lets look at the derivitive w.r.t. <b>var_x</b>:
$$ \frac{\partial \:}{\partial \:x}\left(2x^2 + 3xy\right) = 4x + 3y $$


the derivative w.r.t. <b>var_y</b>:
$$ \frac{\partial \:}{\partial \:x}\left(2x^2 + 3xy\right) = 3x $$

In [153]:
var_grad = tf.gradients(func_test, [var_x, var_y])
session.run(var_grad, {var_x:1.0,var_y:2.0})

[10.0, 3.0]

Now, we can look at gradients w.r.t all variables:

In [157]:
session.run(tf.gradients(cost, tvars), feed_dict)

[IndexedSlicesValue(values=array([[-2.0023101e-06, -6.1247206e-06,  3.4582818e-06, ...,
         -5.7240404e-06,  1.5151845e-05, -9.1785914e-06],
        [ 5.0772805e-06, -3.2938199e-06, -8.3776092e-07, ...,
         -7.8740077e-06,  1.3563555e-05, -1.1688117e-05],
        [ 6.2880436e-06,  1.3215160e-06,  1.9697218e-06, ...,
         -1.1355696e-05,  1.0414182e-05, -9.1363117e-06],
        ...,
        [ 1.2887608e-06,  9.2518212e-07, -2.2497727e-06, ...,
          7.3897686e-06,  7.0581227e-06,  1.6517401e-06],
        [ 2.9282292e-06,  4.3829505e-06,  2.4143583e-06, ...,
          1.0257549e-05,  3.1001343e-06,  6.7647284e-06],
        [ 2.7577994e-06,  8.5802176e-06,  1.8910808e-06, ...,
          5.0415219e-06, -1.6684781e-06, -5.6481963e-07]], dtype=float32), indices=array([9970, 9971, 9972, ..., 2043,   23,    1], dtype=int32), dense_shape=array([10000,   200], dtype=int32)),
 array([[-7.3554617e-08,  1.8004931e-11,  1.4316031e-08, ...,
         -5.9623471e-09,  4.3953779e-08,  




now, we have a list of tensors, t-list. We can use it to find clipped tensors. <b>clip_by_global_norm</b> clips values of multiple tensors by the ratio of the sum of their norms.

<b>clip_by_global_norm</b> get <i>t-list</i> as input and returns 2 things:
<ul>
    <li>a list of clipped tensors, so called <i>list_clipped</i></li> 
    <li>the global norm (global_norm) of all tensors in t_list</li> 
</ul>

In [160]:
# Define the gradient clipping threshold
grads, _ = tf.clip_by_global_norm(tf.gradients(cost, tvars), max_grad_norm)
grads

 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_1:0' shape=(456, 1024) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_2:0' shape=(1024,) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_3:0' shape=(384, 512) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_4:0' shape=(512,) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_5:0' shape=(128, 10000) dtype=float32>,
 <tf.Tensor 'clip_by_global_norm/clip_by_global_norm/_6:0' shape=(10000,) dtype=float32>]

In [161]:
session.run(grads, feed_dict)

[IndexedSlicesValue(values=array([[-2.0023101e-06, -6.1247206e-06,  3.4582818e-06, ...,
         -5.7240404e-06,  1.5151845e-05, -9.1785914e-06],
        [ 5.0772805e-06, -3.2938199e-06, -8.3776092e-07, ...,
         -7.8740077e-06,  1.3563555e-05, -1.1688117e-05],
        [ 6.2880436e-06,  1.3215160e-06,  1.9697218e-06, ...,
         -1.1355696e-05,  1.0414182e-05, -9.1363117e-06],
        ...,
        [ 1.2887608e-06,  9.2518212e-07, -2.2497727e-06, ...,
          7.3897686e-06,  7.0581227e-06,  1.6517401e-06],
        [ 2.9282292e-06,  4.3829505e-06,  2.4143583e-06, ...,
          1.0257549e-05,  3.1001343e-06,  6.7647284e-06],
        [ 2.7577994e-06,  8.5802176e-06,  1.8910808e-06, ...,
          5.0415219e-06, -1.6684781e-06, -5.6481963e-07]], dtype=float32), indices=array([9970, 9971, 9972, ..., 2043,   23,    1], dtype=int32), dense_shape=array([10000,   200], dtype=int32)),
 array([[-7.3554617e-08,  1.8004931e-11,  1.4316031e-08, ...,
         -5.9623471e-09,  4.3953779e-08,  

<h4>4. Apply the optimizer to the variables / gradients tuple.</h4>

In [162]:
# Create the training TensorFlow Operation through our optimizer
train_op = optimizer.apply_gradients(zip(grads, tvars))

In [163]:
session.run(tf.global_variables_initializer())
session.run(train_op, feed_dict)

<a id="ltsm"></a>
<h2>LSTM</h2>

We learned how the model is build step by step. Noe, let's then create a Class that represents our model. This class needs a few things:
<ul>
    <li>We have to create the model in accordance with our defined hyperparameters</li>
    <li>We have to create the placeholders for our input data and expected outputs (the real data)</li>
    <li>We have to create the LSTM cell structure and connect them with our RNN structure</li>
    <li>We have to create the word embeddings and point them to the input data</li>
    <li>We have to create the input structure for our RNN</li>
    <li>We have to instantiate our RNN model and retrieve the variable in which we should expect our outputs to appear</li>
    <li>We need to create a logistic structure to return the probability of our words</li>
    <li>We need to create the loss and cost functions for our optimizer to work, and then create the optimizer</li>
    <li>And finally, we need to create a training operation that can be run to actually train our model</li>
</ul>

In [165]:
class PTBModel(object):

    def __init__(self, action_type):
        ######################################
        # Setting parameters for ease of use #
        ######################################
        self.batch_size = batch_size
        self.num_steps = num_steps
        self.hidden_size_l1 = hidden_size_l1
        self.hidden_size_l2 = hidden_size_l2
        self.vocab_size = vocab_size
        self.embeding_vector_size = embeding_vector_size
        ###############################################################################
        # Creating placeholders for our input data and expected outputs (target data) #
        ###############################################################################
        self._input_data = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]
        self._targets = tf.placeholder(tf.int32, [batch_size, num_steps]) #[30#20]

        ##########################################################################
        # Creating the LSTM cell structure and connect it with the RNN structure #
        ##########################################################################
        # Create the LSTM unit. 
        # This creates only the structure for the LSTM and has to be associated with a RNN unit still.
        # The argument n_hidden(size=200) of BasicLSTMCell is size of hidden layer, that is, the number of hidden units of the LSTM (inside A).
        # Size is the same as the size of our hidden layer, and no bias is added to the Forget Gate. 
        # LSTM cell processes one word at a time and computes probabilities of the possible continuations of the sentence.
        lstm_cell_l1 = tf.contrib.rnn.BasicLSTMCell(self.hidden_size_l1, forget_bias=0.0)
        lstm_cell_l2 = tf.contrib.rnn.BasicLSTMCell(self.hidden_size_l2, forget_bias=0.0)
        
        # Unless you changed keep_prob, this won't actually execute -- this is a dropout wrapper for our LSTM unit
        # This is an optimization of the LSTM output, but is not needed at all
        if action_type == "is_training" and keep_prob < 1:
            lstm_cell_l1 = tf.contrib.rnn.DropoutWrapper(lstm_cell_l1, output_keep_prob=keep_prob)
            lstm_cell_l2 = tf.contrib.rnn.DropoutWrapper(lstm_cell_l2, output_keep_prob=keep_prob)
        
        # By taking in the LSTM cells as parameters, the MultiRNNCell function junctions the LSTM units to the RNN units.
        # RNN cell composed sequentially of multiple simple cells.
        stacked_lstm = tf.contrib.rnn.MultiRNNCell([lstm_cell_l1, lstm_cell_l2])

        # Define the initial state, i.e., the model state for the very first data point
        # It initialize the state of the LSTM memory. The memory state of the network is initialized with a vector of zeros and gets updated after reading each word.
        self._initial_state = stacked_lstm.zero_state(batch_size, tf.float32)

        ####################################################################
        # Creating the word embeddings and pointing them to the input data #
        ####################################################################
        with tf.device("/cpu:0"):
            # Create the embeddings for our input data. Size is hidden size.
            embedding = tf.get_variable("embedding", [vocab_size, self.embeding_vector_size])  #[10000x200]
            # Define where to get the data for our embeddings from
            inputs = tf.nn.embedding_lookup(embedding, self._input_data)

        # Unless you changed keep_prob, this won't actually execute -- this is a dropout addition for our inputs
        # This is an optimization of the input processing and is not needed at all
        if action_type == "is_training" and keep_prob < 1:
            inputs = tf.nn.dropout(inputs, keep_prob)

        ############################################
        # Creating the input structure for our RNN #
        ############################################
        # Input structure is 20x[30x200]
        # Considering each word is represended by a 200 dimentional vector, and we have 30 batchs, we create 30 word-vectors of size [30xx2000]
        # inputs = [tf.squeeze(input_, [1]) for input_ in tf.split(1, num_steps, inputs)]
        # The input structure is fed from the embeddings, which are filled in by the input data
        # Feeding a batch of b sentences to a RNN:
        # In step 1,  first word of each of the b sentences (in a batch) is input in parallel.  
        # In step 2,  second word of each of the b sentences is input in parallel. 
        # The parallelism is only for efficiency.  
        # Each sentence in a batch is handled in parallel, but the network sees one word of a sentence at a time and does the computations accordingly. 
        # All the computations involving the words of all sentences in a batch at a given time step are done in parallel. 

        ####################################################################################################
        # Instantiating our RNN model and retrieving the structure for returning the outputs and the state #
        ####################################################################################################
        
        outputs, state = tf.nn.dynamic_rnn(stacked_lstm, inputs, initial_state=self._initial_state)
        #########################################################################
        # Creating a logistic unit to return the probability of the output word #
        #########################################################################
        output = tf.reshape(outputs, [-1, self.hidden_size_l2])
        softmax_w = tf.get_variable("softmax_w", [self.hidden_size_l2, vocab_size]) #[200x1000]
        softmax_b = tf.get_variable("softmax_b", [vocab_size]) #[1x1000]
        logits = tf.matmul(output, softmax_w) + softmax_b
        logits = tf.reshape(logits, [self.batch_size, self.num_steps, vocab_size])
        prob = tf.nn.softmax(logits)
        out_words = tf.argmax(prob, axis=2)
        self._output_words = out_words
        #########################################################################
        # Defining the loss and cost functions for the model's learning to work #
        #########################################################################
            

        # Use the contrib sequence loss and average over the batches
        loss = tf.contrib.seq2seq.sequence_loss(
            logits,
            self.targets,
            tf.ones([batch_size, num_steps], dtype=tf.float32),
            average_across_timesteps=False,
            average_across_batch=True)
    
#         loss = tf.contrib.legacy_seq2seq.sequence_loss_by_example([logits], [tf.reshape(self._targets, [-1])],
#                                                       [tf.ones([batch_size * num_steps])])
        self._cost = tf.reduce_sum(loss)

        # Store the final state
        self._final_state = state

        #Everything after this point is relevant only for training
        if action_type != "is_training":
            return

        #################################################
        # Creating the Training Operation for our Model #
        #################################################
        # Create a variable for the learning rate
        self._lr = tf.Variable(0.0, trainable=False)
        # Get all TensorFlow variables marked as "trainable" (i.e. all of them except _lr, which we just created)
        tvars = tf.trainable_variables()
        # Define the gradient clipping threshold
        grads, _ = tf.clip_by_global_norm(tf.gradients(self._cost, tvars), max_grad_norm)
        # Create the gradient descent optimizer with our learning rate
        optimizer = tf.train.GradientDescentOptimizer(self.lr)
        # Create the training TensorFlow Operation through our optimizer
        self._train_op = optimizer.apply_gradients(zip(grads, tvars))

    # Helper functions for our LSTM RNN class

    # Assign the learning rate for this model
    def assign_lr(self, session, lr_value):
        session.run(tf.assign(self.lr, lr_value))

    # Returns the input data for this model at a point in time
    @property
    def input_data(self):
        return self._input_data


    
    # Returns the targets for this model at a point in time
    @property
    def targets(self):
        return self._targets

    # Returns the initial state for this model
    @property
    def initial_state(self):
        return self._initial_state

    # Returns the defined Cost
    @property
    def cost(self):
        return self._cost

    # Returns the final state for this model
    @property
    def final_state(self):
        return self._final_state
    
    # Returns the final output words for this model
    @property
    def final_output_words(self):
        return self._output_words
    
    # Returns the current learning rate for this model
    @property
    def lr(self):
        return self._lr

    # Returns the training operation defined for this model
    @property
    def train_op(self):
        return self._train_op

With that, the actual structure of our Recurrent Neural Network with Long Short-Term Memory is finished. What remains for us to do is to actually create the methods to run through time -- that is, the <code>run_epoch</code> method to be run at each epoch and a <code>main</code> script which ties all of this together.

What our <code>run_epoch</code> method should do is take our input data and feed it to the relevant operations. This will return at the very least the current result for the cost function.

In [166]:
##########################################################################################################################
# run_one_epoch takes as parameters the current session, the model instance, the data to be fed, and the operation to be run #
##########################################################################################################################
def run_one_epoch(session, m, data, eval_op, verbose=False):

    #Define the epoch size based on the length of the data, batch size and the number of steps
    epoch_size = ((len(data) // m.batch_size) - 1) // m.num_steps
    start_time = time.time()
    costs = 0.0
    iters = 0

    state = session.run(m.initial_state)
    
    #For each step and data point
    for step, (x, y) in enumerate(reader.ptb_iterator(data, m.batch_size, m.num_steps)):
        
        #Evaluate and return cost, state by running cost, final_state and the function passed as parameter
        cost, state, out_words, _ = session.run([m.cost, m.final_state, m.final_output_words, eval_op],
                                     {m.input_data: x,
                                      m.targets: y,
                                      m.initial_state: state})

        #Add returned cost to costs (which keeps track of the total costs for this epoch)
        costs += cost
        
        #Add number of steps to iteration counter
        iters += m.num_steps

        if verbose and step % (epoch_size // 10) == 10:
            print("Itr %d of %d, perplexity: %.3f speed: %.0f wps" % (step , epoch_size, np.exp(costs / iters), iters * m.batch_size / (time.time() - start_time)))

    # Returns the Perplexity rating for us to keep track of how the model is evolving
    return np.exp(costs / iters)


Now, we create the <code>main</code> method to tie everything together. The code here reads the data from the directory, using the <code>reader</code> helper module, and then trains and evaluates the model on both a testing and a validating subset of data.

In [167]:
# Reads the data and separates it into training data, validation data and testing data
raw_data = reader.ptb_raw_data(data_dir)
train_data, valid_data, test_data, _, _ = raw_data

In [168]:
# Initializes the Execution Graph and the Session
with tf.Graph().as_default(), tf.Session() as session:
    initializer = tf.random_uniform_initializer(-init_scale, init_scale)
    
    # Instantiates the model for training
    # tf.variable_scope add a prefix to the variables created with tf.get_variable
    with tf.variable_scope("model", reuse=None, initializer=initializer):
        m = PTBModel("is_training")
        
    # Reuses the trained parameters for the validation and testing models
    # They are different instances but use the same variables for weights and biases, they just don't change when data is input
    with tf.variable_scope("model", reuse=True, initializer=initializer):
        mvalid = PTBModel("is_validating")
        mtest = PTBModel("is_testing")

    #Initialize all variables
    tf.global_variables_initializer().run()

    for i in range(max_epoch):
        # Define the decay for this epoch
        lr_decay = decay ** max(i - max_epoch_decay_lr, 0.0)
        
        # Set the decayed learning rate as the learning rate for this epoch
        m.assign_lr(session, learning_rate * lr_decay)

        print("Epoch %d : Learning rate: %.3f" % (i + 1, session.run(m.lr)))
        
        # Run the loop for this epoch in the training model
        train_perplexity = run_one_epoch(session, m, train_data, m.train_op, verbose=True)
        print("Epoch %d : Train Perplexity: %.3f" % (i + 1, train_perplexity))
        
        # Run the loop for this epoch in the validation model
        valid_perplexity = run_one_epoch(session, mvalid, valid_data, tf.no_op())
        print("Epoch %d : Valid Perplexity: %.3f" % (i + 1, valid_perplexity))
    
    # Run the loop in the testing model to see how effective was our training
    test_perplexity = run_one_epoch(session, mtest, test_data, tf.no_op())
    
    print("Test Perplexity: %.3f" % test_perplexity)

Epoch 1 : Learning rate: 1.000
Itr 10 of 774, perplexity: 4368.122 speed: 1965 wps
Itr 87 of 774, perplexity: 1290.130 speed: 2072 wps
Itr 164 of 774, perplexity: 998.993 speed: 2080 wps
Itr 241 of 774, perplexity: 827.959 speed: 2083 wps
Itr 318 of 774, perplexity: 732.155 speed: 2081 wps
Itr 395 of 774, perplexity: 652.953 speed: 2081 wps
Itr 472 of 774, perplexity: 589.776 speed: 2080 wps
Itr 549 of 774, perplexity: 534.674 speed: 2078 wps
Itr 626 of 774, perplexity: 491.409 speed: 2078 wps
Itr 703 of 774, perplexity: 456.711 speed: 2077 wps
Epoch 1 : Train Perplexity: 431.998
Epoch 1 : Valid Perplexity: 256.321
Epoch 2 : Learning rate: 1.000
Itr 10 of 774, perplexity: 276.335 speed: 2057 wps


KeyboardInterrupt: 

As you can see, the model's perplexity rating drops very quickly after a few iterations. As was elaborated before, <b>lower Perplexity means that the model is more certain about its prediction</b>. As such, we can be sure that this model is performing well!

-------

This is the end of the <b>Applying Recurrent Neural Networks to Text Processing</b> notebook. Hopefully you now have a better understanding of Recurrent Neural Networks and how to implement one utilizing TensorFlow. Thank you for reading this notebook, and good luck on your studies.

## Want to learn more?

Running deep learning programs usually needs a high performance platform. __PowerAI__ speeds up deep learning and AI. Built on IBM’s Power Systems, __PowerAI__ is a scalable software platform that accelerates deep learning and AI with blazing performance for individual users or enterprises. The __PowerAI__ platform supports popular machine learning libraries and dependencies including TensorFlow, Caffe, Torch, and Theano. You can use [PowerAI on IMB Cloud](https://cocl.us/ML0120EN_PAI).

Also, you can use __Watson Studio__ to run these notebooks faster with bigger datasets.__Watson Studio__ is IBM’s leading cloud solution for data scientists, built by data scientists. With Jupyter notebooks, RStudio, Apache Spark and popular libraries pre-packaged in the cloud, __Watson Studio__ enables data scientists to collaborate on their projects without having to install anything. Join the fast-growing community of __Watson Studio__ users today with a free account at [Watson Studio](https://cocl.us/ML0120EN_DSX).This is the end of this lesson. Thank you for reading this notebook, and good luck on your studies.

### Thanks for completing this lesson!

Notebook created by <a href="https://br.linkedin.com/in/walter-gomes-de-amorim-junior-624726121">Walter Gomes de Amorim Junior</a>, <a href = "https://linkedin.com/in/saeedaghabozorgi"> Saeed Aghabozorgi </a></h4>

<hr>

Copyright &copy; 2018 [Cognitive Class](https://cocl.us/DX0108EN_CC). This notebook and its source code are released under the terms of the [MIT License](https://bigdatauniversity.com/mit-license/).